
### How to use a logistic classification regression model to predict Churn on pre-processed Google Analytics 4 data using  BigQueryML

[original source](https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/bigquery_ml/bqml-online-prediction.ipynb)

### Objective

In this tutorial, you learn how to train and deploy a churn prediction model for real-time inference, with the data in BigQuery and model trained using BigQuery ML, registered to Vertex AI Model Registry, and deployed to an endpoint on Vertex AI for online predictions.

This tutorial uses the following Google Cloud data analytics and ML services:

- BigQuery
- BigQuery ML
- Vertex AI Model Registry
- Vertex endpoints


The steps performed include:

- Using Python & SQL to query the public data in BigQuery
- Preparing the data for modeling
- Training a classification model using BigQuery ML and registering it to Vertex AI Model Registry
- Inspecting the model on Vertex AI Model Registry
- Deploying the model to an endpoint on Vertex AI
- Making sample online predictions to the model endpoint


### Dataset

The dataset, <a href="https://console.cloud.google.com/bigquery?project=bigquery-public-data&d=ga4_obfuscated_sample_ecommerce&p=bigquery-public-data&page=dataset" target="_blank">available publicly on BigQuery</a>, comes from obfuscated <a href="https://support.google.com/analytics/answer/10937659" target="_blank">Google Analytics 4 data</a> from the <a href="https://shop.googlemerchandisestore.com/" target="_blank">Google Merchandise Store</a>).

### Install additional packages

Install the following packages required to execute this notebook. 

In [1]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install --upgrade google-cloud-aiplatform {USER_FLAG} -q google-cloud-bigquery db-dtypes

     |████████████████████████████████| 2.3 MB 4.7 MB/s 
     |████████████████████████████████| 211 kB 37.7 MB/s 
     |████████████████████████████████| 233 kB 28.0 MB/s 
     |████████████████████████████████| 408 kB 42.4 MB/s 
     |████████████████████████████████| 106 kB 44.6 MB/s 
     |████████████████████████████████| 47 kB 1.8 MB/s 
     |████████████████████████████████| 115 kB 49.5 MB/s 
     |████████████████████████████████| 206 kB 50.8 MB/s 
     |████████████████████████████████| 77 kB 3.2 MB/s 
     |████████████████████████████████| 1.0 MB 50.0 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.9.2 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.
tensorboard 2.9.1 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.
pandas-gbq 0.13.3 requires google-cloud-bigquer

### Restart the kernel

After you install the additional packages, you need to restart the notebook kernel so it can find the packages.

In [2]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Project Variables 

In [4]:
# Project variables 
#
# These are the project variable used in this ML Model: 
#

PROJECT_ID = "" # @param {type:"string"}
bqml_type = "log_reg" # @param {type:"string"}
BQML_MODEL_NAME = "bqml_log_reg_analytics_churn_predict_model"
job_display_name = BQML_MODEL_NAME + "_job"
ENDPOINT_NAME = BQML_MODEL_NAME + "_endpoint"

# datasets
BQ_DATASET_NAME = BQML_MODEL_NAME + "_dataset"
sql_create_dataset = f"""CREATE SCHEMA IF NOT EXISTS {BQ_DATASET_NAME}"""

# bucket details
BUCKET_NAME = "bqml_tutorials"
BUCKET_URI = f"gs://{BUCKET_NAME}/{bqml_type}/"
OUTPUTBUCKET = f"gs://bqml_datasets_predictions/{bqml_type}/"

# Region 
REGION = "us-central1" # @param {type: "string"} 

Otherwise, set your project ID here.

#### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook.  Below are regions supported for Vertex AI. We recommend that you choose the region closest to you.

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

You might not be able to use a multi-regional bucket for training with Vertex AI. Not all regions provide support for all Vertex AI services.

Learn more about <a href="https://cloud.google.com/vertex-ai/docs/general/locations" target="_blank">Vertex AI regions</a>.

In [5]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = "google.colab" in sys.modules
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

In [11]:
SERVICE_ACCOUNT = ""  # @param {type:"string"}
print(SERVICE_ACCOUNT)

In [ ]:
if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your service account from gcloud
    if not IS_COLAB:
        shell_output = !gcloud auth list 2>/dev/null
        SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

    else:  # IS_COLAB:
        shell_output = ! gcloud projects describe  $PROJECT_ID
        project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
        SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

    print("Service Account:", SERVICE_ACCOUNT)

### Import libraries

In [13]:
from typing import Union

import google.cloud.aiplatform as vertex_ai
import pandas as pd
from google.cloud import bigquery

### Initialize Vertex AI and BigQuery SDKs for Python

Initialize the Vertex AI SDK for Python for your project and corresponding bucket.

In [14]:
vertex_ai.init(project=PROJECT_ID, location=REGION)

Create the BigQuery client.

In [15]:
bq_client = bigquery.Client(project=PROJECT_ID)

Use a helper function for sending queries to BigQuery.

In [16]:
# Wrapper to use BigQuery client to run query/job, return job ID or result as DF
def run_bq_query(sql: str) -> Union[str, pd.DataFrame]:
    """
    Input: SQL query, as a string, to execute in BigQuery
    Returns the query results as a pandas DataFrame, or error, if any
    """

    # Try dry run before executing query to catch any errors
    job_config = bigquery.QueryJobConfig(dry_run=True, use_query_cache=False)
    bq_client.query(sql, job_config=job_config)

    # If dry run succeeds without errors, proceed to run query
    job_config = bigquery.QueryJobConfig()
    client_result = bq_client.query(sql, job_config=job_config)

    job_id = client_result.job_id

    # Wait for query/job to finish running. then get & return data frame
    df = client_result.result().to_arrow().to_pandas()
    print(f"Finished job_id: {job_id}")
    return df

## BigQuery ML Model Training & Validation

BigQuery ML (BQML) provides the capability to train ML tabular models, such as classification, regression, forecasting, and matrix factorization, in BigQuery using SQL syntax directly. BigQuery ML uses the scalable infrastructure of BigQuery ML so you don't need to set up additional infrastructure for training or batch serving.

In [17]:
sql_create_dataset = f"""CREATE SCHEMA IF NOT EXISTS {BQ_DATASET_NAME}"""

print(sql_create_dataset)

run_bq_query(sql_create_dataset)

CREATE SCHEMA IF NOT EXISTS bqml_log_reg_analytics_churn_predict_model_dataset
Finished job_id: fdcba794-9e6a-4e5a-9f40-04344e1896e6


""


### Using logistic regression model for classification on pre-processed Google Analytics 4 data

Inpect data that has been pre-processed from <a href="https://support.google.com/analytics/answer/10937659" target="_blank">Google Analytics 4 data from the Google Merchandise Store</a> so that it can be used for classification. For more information on how this data was prepared, read <a href="https://cloud.google.com/blog/topics/developers-practitioners/churn-prediction-game-developers-using-google-analytics-4-ga4-and-bigquery-ml" target="_blank">this blog post</a>.

As seen below, each row represents a single user, and the columns represent their demographic features, their aggregated behavioral features in the first 24 hours of visiting the Google Merchandise Store, and the label (whether the user churned or returned any time after the first 24 hours).

In [18]:
sql_inspect = """
SELECT
    *
FROM
    `bqmlpublic.demo_ga4churnprediction.training_data`
LIMIT
    100
"""
run_bq_query(sql_inspect)

/usr/local/lib/python3.7/dist-packages/google/cloud/bigquery/table.py:1567: UserWarning: Dependency google-cloud-bigquery-storage is outdated, please upgrade it to version >= 2.0.0 (version found: 0.0.0).
  warnings.warn(str(exc))


Finished job_id: 813ddabc-e3a3-4819-8141-0be27af2ced0


,user_pseudo_id,country,operating_system,language,cnt_user_engagement,cnt_page_view,cnt_view_item,cnt_view_promotion,cnt_select_promotion,cnt_add_to_cart,cnt_begin_checkout,cnt_add_shipping_info,cnt_add_payment_info,cnt_purchase,user_first_engagement,month,julianday,dayofweek,churned
0,7666337.2408476627,United States,Web,en-us,5,6,2,0,0,0,0,0,0,0,2021-01-01 19:23:47.339545+00:00,1,1,6,1
1,44740556.1615757380,Canada,Macintosh,None,6,7,2,0,0,0,0,0,0,0,2021-01-01 11:42:53.072756+00:00,1,1,6,1
2,91028872.0097105946,United States,Windows,en-us,2,3,0,1,0,0,0,0,0,0,2021-01-01 19:07:53.669542+00:00,1,1,6,1
3,63655374.9348413503,United States,Web,es-es,8,10,6,3,0,0,0,0,0,0,2021-01-01 12:03:29.281546+00:00,1,1,6,0
4,10115718.4867090359,United States,Web,None,15,12,9,0,0,0,0,0,0,0,2021-01-01 18:32:29.296973+00:00,1,1,6,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2304235.4728947355,United States,Web,en-us,1,2,0,0,0,0,0,0,0,0,2021-01-01 16:39:41.910818+00:00,1,1,6,0
96,4902634.1944567521,Germany,Web,en-us,1,3,0,1,0,0,0,0,0,0,2021-01-01 12:08:53.020090+00:00,1,1,6,0
97,6385315.9680167296,Russia,Web,en-us,2,5,1,2,1,0,0,0,0,0,2021-01-01 11:08:32.462414+00:00,1,1,6,1
98,36665674.3318230125,Spain,<Other>,en-us,2,4,0,0,0,0,0,0,0,0,2021-01-01 03:06:33.897251+00:00,1,1,6,1


### Train a classification model using BigQuery ML

The query below trains a logistic regression model using BigQuery ML. BigQuery resources are used to train the model.

In the `OPTIONS` parameter:
* with `model_registry="vertex_ai"`, the BigQuery ML model will automatically be <a href="https://cloud.google.com/vertex-ai/docs/model-registry/model-registry-bqml" target="_blank">registered to Vertex AI Model Registry</a>, which enables you to view all of your registered models and its versions on Google Cloud in one place.

* `vertex_ai_model_version_aliases allows you to set aliases to help you keep track of your model version (<a href="https://cloud.google.com/vertex-ai/docs/model-registry/model-alias" target="_blank">documentation</a>).

In [19]:
# this cell may take ~1 min to run

sql_train_model_bqml = f"""
CREATE OR REPLACE MODEL {BQ_DATASET_NAME}.{BQML_MODEL_NAME}    
OPTIONS(
  MODEL_TYPE="LOGISTIC_REG",
  input_label_cols=["churned"],
  model_registry="vertex_ai",
  vertex_ai_model_version_aliases=['logistic_reg', 'experimental']
) AS

SELECT
  * EXCEPT(user_first_engagement, user_pseudo_id)
FROM
  bqmlpublic.demo_ga4churnprediction.training_data
"""

print(sql_train_model_bqml)

run_bq_query(sql_train_model_bqml)


CREATE OR REPLACE MODEL bqml_log_reg_analytics_churn_predict_model_dataset.bqml_log_reg_analytics_churn_predict_model    
OPTIONS(
  MODEL_TYPE="LOGISTIC_REG",
  input_label_cols=["churned"],
  model_registry="vertex_ai",
  vertex_ai_model_version_aliases=['logistic_reg', 'experimental']
) AS

SELECT
  * EXCEPT(user_first_engagement, user_pseudo_id)
FROM
  bqmlpublic.demo_ga4churnprediction.training_data

Finished job_id: c541796c-d0e5-4ff8-85d7-d1e7626e1749


""


### Model evaluation

With the model created, you can now evaluate the logistic regression model. Behind the scenes, BigQuery ML automatically <a href="https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create#data_split_method" target="_blank">split the data</a>, which makes it easier to quickly train and evaluate models.

In [20]:
sql_evaluate_model = f"""
SELECT
  *
FROM
  ML.EVALUATE(MODEL {BQ_DATASET_NAME}.{BQML_MODEL_NAME})
"""

print(sql_evaluate_model)

run_bq_query(sql_evaluate_model)


SELECT
  *
FROM
  ML.EVALUATE(MODEL bqml_log_reg_analytics_churn_predict_model_dataset.bqml_log_reg_analytics_churn_predict_model)



/usr/local/lib/python3.7/dist-packages/google/cloud/bigquery/table.py:1567: UserWarning: Dependency google-cloud-bigquery-storage is outdated, please upgrade it to version >= 2.0.0 (version found: 0.0.0).
  warnings.warn(str(exc))


Finished job_id: b62e58a5-390e-4cc2-945f-1edf077e0f67


,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,0.571924,0.995775,0.571083,0.726552,0.671342,0.602277


These metrics help you understand the performance of the model. 

There are various metrics for logistic regression and other model types (full list of metrics can be found in the <a href="https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-evaluate#mlevaluate_output" target="_blank">documentation</a>).

### Batch prediction (with Explainable AI)

Make a batch prediction in BigQuery ML on the original training data to check the probability of churn for each of the users, as seen in the `probability` column, with the predicted label under the `predicted_churn` column.

<a href="https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-explain-predict" target="_blank">ML.EXPLAIN_PREDICT</a> has built-in <a href="https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-xai-overview" target="_blank">Explainable AI</a>. This allows you to see the top contributing features to each prediction and interpret how it was computed.

In [21]:
sql_explain_predict = f"""
SELECT
  *
FROM
  ML.EXPLAIN_PREDICT(MODEL {BQ_DATASET_NAME}.{BQML_MODEL_NAME},
    (SELECT * FROM bqmlpublic.demo_ga4churnprediction.training_data LIMIT 100)
    )
"""

print(sql_explain_predict)

run_bq_query(sql_explain_predict)


SELECT
  *
FROM
  ML.EXPLAIN_PREDICT(MODEL bqml_log_reg_analytics_churn_predict_model_dataset.bqml_log_reg_analytics_churn_predict_model,
    (SELECT * FROM bqmlpublic.demo_ga4churnprediction.training_data LIMIT 100)
    )



/usr/local/lib/python3.7/dist-packages/google/cloud/bigquery/table.py:1567: UserWarning: Dependency google-cloud-bigquery-storage is outdated, please upgrade it to version >= 2.0.0 (version found: 0.0.0).
  warnings.warn(str(exc))


Finished job_id: 57e7b60b-4928-48f1-8299-26cf9656f234


,predicted_churned,probability,top_feature_attributions,baseline_prediction_value,prediction_value,approximation_error,user_pseudo_id,country,operating_system,language,...,cnt_add_to_cart,cnt_begin_checkout,cnt_add_shipping_info,cnt_add_payment_info,cnt_purchase,user_first_engagement,month,julianday,dayofweek,churned
0,1,0.590461,"[{'feature': 'language', 'attribution': 0.0975...",0.069157,0.365871,0.0,7666337.2408476627,United States,Web,en-us,...,0,0,0,0,0,2021-01-01 19:23:47.339545+00:00,1,1,6,1
1,1,0.564491,"[{'feature': 'month', 'attribution': 0.0808542...",0.069157,0.259407,0.0,44740556.1615757380,Canada,Macintosh,None,...,0,0,0,0,0,2021-01-01 11:42:53.072756+00:00,1,1,6,1
2,1,0.591651,"[{'feature': 'language', 'attribution': 0.0975...",0.069157,0.370796,0.0,91028872.0097105946,United States,Windows,en-us,...,0,0,0,0,0,2021-01-01 19:07:53.669542+00:00,1,1,6,1
3,1,0.612900,"[{'feature': 'month', 'attribution': 0.0808542...",0.069157,0.459517,0.0,63655374.9348413503,United States,Web,es-es,...,0,0,0,0,0,2021-01-01 12:03:29.281546+00:00,1,1,6,0
4,1,0.595990,"[{'feature': 'month', 'attribution': 0.0808542...",0.069157,0.388785,0.0,10115718.4867090359,United States,Web,None,...,0,0,0,0,0,2021-01-01 18:32:29.296973+00:00,1,1,6,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1,0.582265,"[{'feature': 'language', 'attribution': 0.0975...",0.069157,0.332077,0.0,2304235.4728947355,United States,Web,en-us,...,0,0,0,0,0,2021-01-01 16:39:41.910818+00:00,1,1,6,0
96,1,0.595478,"[{'feature': 'country', 'attribution': 0.09822...",0.069157,0.386659,0.0,4902634.1944567521,Germany,Web,en-us,...,0,0,0,0,0,2021-01-01 12:08:53.020090+00:00,1,1,6,0
97,1,0.594728,"[{'feature': 'language', 'attribution': 0.0975...",0.069157,0.383546,0.0,6385315.9680167296,Russia,Web,en-us,...,0,0,0,0,0,2021-01-01 11:08:32.462414+00:00,1,1,6,1
98,1,0.578454,"[{'feature': 'language', 'attribution': 0.0975...",0.069157,0.316432,0.0,36665674.3318230125,Spain,<Other>,en-us,...,0,0,0,0,0,2021-01-01 03:06:33.897251+00:00,1,1,6,1


Since the `top_feature_attributions` is a nested column, you can unnest the array (<a href="https://cloud.google.com/bigquery/docs/reference/standard-sql/arrays" target="_blank">documentation</a>) into separate rows for each of the features. In other words, since ML.EXPLAIN_PREDICT provides the top 5 most important features, using `UNNEST` results in 5 rows per prediction:

In [22]:
sql_explain_predict = f"""
SELECT
  tfa.*,
  predicted_churned,
  probability,
  baseline_prediction_value,
  prediction_value,
  approximation_error,
  user_pseudo_id
FROM
  ML.EXPLAIN_PREDICT(MODEL {BQ_DATASET_NAME}.{BQML_MODEL_NAME},
    (SELECT * FROM bqmlpublic.demo_ga4churnprediction.training_data LIMIT 100)
    ),
  UNNEST(top_feature_attributions) as tfa
WHERE
  user_pseudo_id = "7666337.2408476627"
"""

print(sql_explain_predict)

run_bq_query(sql_explain_predict)


SELECT
  tfa.*,
  predicted_churned,
  probability,
  baseline_prediction_value,
  prediction_value,
  approximation_error,
  user_pseudo_id
FROM
  ML.EXPLAIN_PREDICT(MODEL bqml_log_reg_analytics_churn_predict_model_dataset.bqml_log_reg_analytics_churn_predict_model,
    (SELECT * FROM bqmlpublic.demo_ga4churnprediction.training_data LIMIT 100)
    ),
  UNNEST(top_feature_attributions) as tfa
WHERE
  user_pseudo_id = "7666337.2408476627"



/usr/local/lib/python3.7/dist-packages/google/cloud/bigquery/table.py:1567: UserWarning: Dependency google-cloud-bigquery-storage is outdated, please upgrade it to version >= 2.0.0 (version found: 0.0.0).
  warnings.warn(str(exc))


Finished job_id: ee766da5-ed08-460f-9b0c-9869ebbd9c53


,feature,attribution,predicted_churned,probability,baseline_prediction_value,prediction_value,approximation_error,user_pseudo_id
0,language,0.097598,1,0.590461,0.069157,0.365871,0.0,7666337.2408476627
1,month,0.080854,1,0.590461,0.069157,0.365871,0.0,7666337.2408476627
2,julianday,0.080515,1,0.590461,0.069157,0.365871,0.0,7666337.2408476627
3,operating_system,0.075108,1,0.590461,0.069157,0.365871,0.0,7666337.2408476627
4,country,0.074942,1,0.590461,0.069157,0.365871,0.0,7666337.2408476627


### Inspect the model on Vertex AI Model Registry

When the model was trained in BigQuery ML, the line `model_registry="vertex_ai"` registered the model to Vertex AI Model Registry automatically upon completion.

You can view the model on the <a href="https://console.cloud.google.com/vertex-ai/models" target="_blank">Vertex AI Model Registry page</a>, or use the code below to check that it was successfully registered:

In [23]:
model = vertex_ai.Model(model_name=BQML_MODEL_NAME)

print(model.gca_resource)

name: "projects/993987777814/locations/us-central1/models/bqml_log_reg_analytics_churn_predict_model"
display_name: "bqml_log_reg_analytics_churn_predict_model"
supported_deployment_resources_types: DEDICATED_RESOURCES
supported_input_storage_formats: "jsonl"
supported_input_storage_formats: "bigquery"
supported_input_storage_formats: "csv"
supported_input_storage_formats: "tf-record"
supported_input_storage_formats: "tf-record-gzip"
supported_input_storage_formats: "file-list"
supported_output_storage_formats: "jsonl"
supported_output_storage_formats: "bigquery"
create_time {
  seconds: 1667530532
  nanos: 922262000
}
update_time {
  seconds: 1667530556
  nanos: 13847000
}
etag: "AMEw9yP-ipPaBHHrDblBPWE0LPQWMkrL0U7TfzDW1XAIejpMpHpzQqLXZaiDqNm75JQ="
version_id: "1"
version_aliases: "logistic_reg"
version_aliases: "experimental"
version_aliases: "default"
version_create_time {
  seconds: 1667530532
  nanos: 922262000
}
version_update_time {
  seconds: 1667530555
  nanos: 974423000
}
mod

### Deploy the model to an endpoint

While BigQuery ML supports batch prediction with <a href="https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-predict" target="_blank">ML.PREDICT</a> and <a href="https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-explain-predict" target="_blank">ML.EXPLAIN_PREDICT</a>, BigQuery ML is not suitable for real-time predictions where you need low latency predictions with potentially high frequency of requests.

In other words, deploying the BigQuery ML model to an endpoint enables you to do online predictions.

#### Create a Vertex AI endpoint

To deploy your model to an endpoint, you will first need to create an endpoint before you deploy the model to it.

In [24]:

endpoint = vertex_ai.Endpoint.create(
    display_name=ENDPOINT_NAME,
    project=PROJECT_ID,
    location=REGION,
)

print(endpoint.display_name)
print(endpoint.resource_name)

Creating Endpoint


INFO:google.cloud.aiplatform.models:Creating Endpoint


Create Endpoint backing LRO: projects/993987777814/locations/us-central1/endpoints/4502689231742697472/operations/453959007892144128


INFO:google.cloud.aiplatform.models:Create Endpoint backing LRO: projects/993987777814/locations/us-central1/endpoints/4502689231742697472/operations/453959007892144128


Endpoint created. Resource name: projects/993987777814/locations/us-central1/endpoints/4502689231742697472


...

To use this Endpoint in another session:


INFO:google.cloud.aiplatform.models:To use this Endpoint in another session:


endpoint = aiplatform.Endpoint('projects/993987777814/locations/us-central1/endpoints/4502689231742697472')


INFO:google.cloud.aiplatform.models:endpoint = aiplatform.Endpoint('projects/993987777814/locations/us-central1/endpoints/4502689231742697472')


bqml_log_reg_analytics_churn_predict_model_endpoint
projects/993987777814/locations/us-central1/endpoints/4502689231742697472


#### List endpoints

List the endpoints to make sure it has successfully been created. (You can also view your endpoints on the <a href="https://console.cloud.google.com/vertex-ai/endpoints" target="_blank">Vertex AI Endpoints page</a>).

In [25]:
endpoint.list()

 resource name: projects/993987777814/locations/us-central1/endpoints/4502689231742697472,
 resource name: projects/993987777814/locations/us-central1/endpoints/369919692048957440,
 resource name: projects/993987777814/locations/us-central1/endpoints/1236862620317777920,
 resource name: projects/993987777814/locations/us-central1/endpoints/5420706674144968704,
 resource name: projects/993987777814/locations/us-central1/endpoints/7994667792815095808]

#### Deploy model to Vertex endpoint

With the new endpoint, you can now deploy your model.

In [26]:
# deploying the model to the endpoint may take 10-15 minutes
model.deploy(endpoint=endpoint)

Deploying model to Endpoint : projects/993987777814/locations/us-central1/endpoints/4502689231742697472


INFO:google.cloud.aiplatform.models:Deploying model to Endpoint : projects/993987777814/locations/us-central1/endpoints/4502689231742697472


Using default machine_type: n1-standard-2


INFO:google.cloud.aiplatform.models:Using default machine_type: n1-standard-2


Deploy Endpoint model backing LRO: projects/993987777814/locations/us-central1/endpoints/4502689231742697472/operations/6681874342592118784


INFO:google.cloud.aiplatform.models:Deploy Endpoint model backing LRO: projects/993987777814/locations/us-central1/endpoints/4502689231742697472/operations/6681874342592118784


Endpoint model deployed. Resource name: projects/993987777814/locations/us-central1/endpoints/4502689231742697472


INFO:google.cloud.aiplatform.models:Endpoint model deployed. Resource name: projects/993987777814/locations/us-central1/endpoints/4502689231742697472


resource name: projects/993987777814/locations/us-central1/endpoints/4502689231742697472

You can also check on the status of your model by visiting the <a href="https://console.cloud.google.com/vertex-ai/endpoints" target="_blank">Vertex AI Endpoints page</a>.

### Make online predictions to the endpoint

Using a sample of the training data, you can test the endpoint to make online predictions.

In [27]:
df_sample_requests_list = [
    {
        "country": "Turkey",
        "operating_system": "Web",
        "language": "None",
        "cnt_user_engagement": 28,
        "cnt_page_view": 37,
        "cnt_view_item": 6,
        "cnt_view_promotion": 15,
        "cnt_select_promotion": 4,
        "cnt_add_to_cart": 0,
        "cnt_begin_checkout": 0,
        "cnt_add_shipping_info": 0,
        "cnt_add_payment_info": 0,
        "cnt_purchase": 0,
        "month": 1,
        "julianday": 1,
        "dayofweek": 6,
    },
    {
        "country": "Macao",
        "operating_system": "Web",
        "language": "None",
        "cnt_user_engagement": 2,
        "cnt_page_view": 4,
        "cnt_view_item": 0,
        "cnt_view_promotion": 0,
        "cnt_select_promotion": 0,
        "cnt_add_to_cart": 0,
        "cnt_begin_checkout": 0,
        "cnt_add_shipping_info": 0,
        "cnt_add_payment_info": 0,
        "cnt_purchase": 0,
        "month": 1,
        "julianday": 16,
        "dayofweek": 7,
    },
]

In [28]:
prediction = endpoint.predict(df_sample_requests_list)
print(prediction)

Prediction(predictions=[{'churned_values': ['1', '0'], 'churned_probs': [0.7541578275932658, 0.2458421724067342], 'predicted_churned': ['1']}, {'predicted_churned': ['0'], 'churned_values': ['1', '0'], 'churned_probs': [0.4827901063200341, 0.517209893679966]}], deployed_model_id='3072145439168921600', model_version_id='1', model_resource_name='projects/993987777814/locations/us-central1/models/bqml_log_reg_analytics_churn_predict_model', explanations=None)


You can then extract the predictions from the prediction response

In [29]:
prediction.predictions

[{'churned_values': ['1', '0'],
  'churned_probs': [0.7541578275932658, 0.2458421724067342],
  'predicted_churned': ['1']},
 {'predicted_churned': ['0'],
  'churned_values': ['1', '0'],
  'churned_probs': [0.4827901063200341, 0.517209893679966]}]

## Cleaning up

To clean up all Google Cloud resources used in this project, you can <a href="https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects" target="_blank">delete the Google Cloud
project</a> you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

In [ ]:
# Undeploy model from endpoint and delete endpoint
endpoint.undeploy_all()
endpoint.delete()

# Delete BigQuery dataset, including the BigQuery ML model
! bq rm -r -f $PROJECT_ID:$BQ_DATASET_NAME